## Arbol elemental con libreria  rpart

Debe tener instaladas las librerias  data.table  ,  rpart   y rpart.plot

In [1]:
#cargo las librerias que necesito
require("data.table")
require("rpart")
require("rpart.plot")

Loading required package: data.table

Loading required package: rpart

Loading required package: rpart.plot



Aqui se debe poner la carpeta de SU computadora local

In [2]:
setwd("C:\\Users\\agust\\Desktop\\laboimplementacion")  #Establezco el Working Directory

cargo el dataset

In [9]:
dataset  <- fread("./datasets/dataset_pequeno.csv")



[1] "data.table" "data.frame"

defino donde entreno y donde aplico el modelo

In [ ]:
dtrain  <- dataset[ foto_mes==202107 ]  #defino donde voy a entrenar
dapply  <- dataset[ foto_mes==202109 ]  #defino donde voy a aplicar el modelo

genero el modelo,  aqui se construye el arbol

In [ ]:
modelo  <- rpart(formula=    "clase_ternaria ~ .", #quiero predecir clase_ternaria a partir de el resto de las variables
                 data =      dtrain,  #los datos donde voy a entrenar
                 xval=       0,
                 cp=        -0.3, #esto significa no limitar la complejidad de los splits
                 minsplit= 100,   #minima cantidad de registros para que se haga el split
                 minbucket=  1,   #tamaño minimo de una hoja
                 maxdepth=  10 )  #PRUEBO  una profundidad de 10

Ahora grafico el arbol, va a ser muy frondoso porque la profundidad es de 10

In [ ]:
prp(modelo, extra=101, digits=-5, branch=1, type=4, varlen=0, faclen=0)

Ahora aplico al modelo  a los datos muevos  y genero la salida para kaggle

aplico el modelo a los datos nuevos

In [ ]:
prediccion  <- predict( modelo, dapply , type = "prob") #aplico el modelo

prediccion es una matriz con TRES columnas, llamadas "BAJA+1", "BAJA+2"  y "CONTINUA"
cada columna es el vector de probabilidades 

agrego a dapply una columna nueva que es la probabilidad de BAJA+2

In [ ]:
dapply[ , prob_baja2 := prediccion[, "BAJA+2"] ]

solo le envio estimulo a los registros con probabilidad de BAJA+2 mayor  a  1/40

In [ ]:
dapply[ , Predicted  := as.numeric(prob_baja2 > 1/40) ]

Finalmente genero el archivo para Kaggle

creo la carpeta donde va el experimento

In [ ]:
dir.create( "./exp/KA2002" ) 

grabo el dataset

In [ ]:
fwrite( dapply[ , list(numero_de_cliente, Predicted) ], #solo los campos para Kaggle
        file= "./exp/KA2002/KA2002_001.csv", 
        sep= "," )